In [1]:
#read data from csv
import sys
sys.path.append('..')
import logging
logging.basicConfig(level=logging.WARNING)

from src.llm_alex import Llama
from langchain_core import pydantic_v1
from langchain_core.runnables.base import RunnableParallel, RunnableLambda
from langchain.output_parsers.retry import RetryOutputParser
from langchain_core.output_parsers import JsonOutputParser
from typing import List

from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator

from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate



import numpy
import pandas as pd
import random 
import tqdm 
import time
import wandb

from sklearn.metrics import classification_report

train_data = pd.read_csv("data/train_data.csv")
test_data = pd.read_csv("data/test_data.csv")

llm = Llama()

ImportError: huggingface-hub>=0.23.0,<1.0 is required for a normal functioning of this module, but found huggingface-hub==0.22.2.
Try: `pip install transformers -U` or `pip install -e '.[dev]'` if you're working with git main

In [ ]:
random_seed = 42
n_classes = 2
num_shots = 2
sampled_classes = train_data['label'].sample(n_classes, random_state=random_seed).values
train_data_sub = train_data[train_data['label'].isin(sampled_classes)]
test_data_sub = test_data[test_data['label'].isin(sampled_classes)]

def get_prompt_template(train_data_sub, num_shots):
    prompt_template = "Your task is to classify a given text into one of the following classes, reply ONLY with the class: \n\n"
    for label in train_data_sub.label.unique():
        prompt_template += f"Label: {label}\n"
        for i, row in train_data_sub[train_data_sub['label'] == label].sample(num_shots, random_state=random_seed).iterrows():
            prompt_template += f"Text: {row['text']}\n"
            #remove row from the dataframe
            train_data_sub = train_data_sub.drop(i)
        prompt_template += "\n"
    prompt_template += "Here is your text, please classify it into one of the above classes\n\n"
    return prompt_template

In [3]:
#prompt_template = get_prompt_template(test_data_sub,num_shots)

text = get_prompt_template(test_data_sub,num_shots)
text += "Text: my credit card does not work"

In [4]:
print(text)

Your task is to classify a given text into one of the following classes, reply ONLY with the class: 

Label: 25
Text: Why are you declining my payment? Everything was fine.
Text: I have a card payment that was declined, but why?

Label: 21
Text: I would like to change my pin.
Text: How can I change my Tholepin ?

Here is your text, please classify it into one of the above classes

Text: my credit card does not work


In [5]:
response = llm(query=prompt)
response

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'21'

In [8]:
# Define your desired data structure.
class Prediction(BaseModel):
    label: int = Field(description="classification label to a given text")

    # You can add custom validation logic easily with Pydantic.
    @validator("label")
    def question_ends_with_question_mark(cls, field):
        #TODO: validation error not a number anpassen
        #if field[-1] != "?":
         #   raise ValueError("Badly formed question!")
        return field
parser = PydanticOutputParser(pydantic_object=Prediction)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

retry_parser = RetryOutputParser.from_llm(parser=parser, llm=llm, max_retries=3)
 
completion_chain = prompt | llm

main_chain = RunnableParallel(
    completion=completion_chain, prompt_value=prompt
) | RunnableLambda(lambda x: retry_parser.parse_with_prompt(**x))


answer: Prediction = main_chain.invoke({"query": text})
#answer_final: str = answer.label
#reasoning_final: str = answer.reasoning

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


In [11]:
type(answer.label)

int

### TODO:
- 

In [12]:
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"label": {"title": "Label", "description": "classification label to a given text", "type": "integer"}}, "required": ["label"]}\n```'

In [25]:

for i, row in tqdm.tqdm(test_data_sub.iterrows(),total=len(test_data_sub), desc="Processing data"):
    prompt = prompt_template + f"Text: {row['text']}\n"
    response = llm(prompt=prompt)
    test_data_sub.loc[i, 'response'] = response
    #print(f"Prompt: {prompt}")
    #print(f"Response: {response}")
    print("\n\n")

Processing data:   0%|          | 0/80 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
/scratch/slurm_tmpdir/job_23779533/ipykernel_169380/1091179139.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data_sub.loc[i, 'response'] = response
Processing data:   1%|▏         | 1/80 [00:16<21:17, 16.18s/it]Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Processing data:   2%|▎         | 2/80 [00:16<09:01,  6.94s/it]Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Processing data:   4%|▍         | 3/80 [00:17<05:06,  3.98s/it]Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


...

Processing data:  95%|█████████▌| 76/80 [00:51<00:01,  2.14it/s]Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Processing data:  96%|█████████▋| 77/80 [00:51<00:01,  2.14it/s]Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Processing data:  98%|█████████▊| 78/80 [00:52<00:00,  2.14it/s]Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Processing data:  99%|█████████▉| 79/80 [00:52<00:00,  2.06it/s]Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Processing data: 100%|██████████| 80/80 [00:53<00:00,  1.51it/s]

In [26]:
test_data_sub

,text,label,response
40,Why won't my card show up on the app?,13,13
41,I would like to reactivate my card.,13,13
42,Where do I link the new card?,13,13
43,"I have received my card, can you help me put i...",13,13
44,How do I link a card that I already have?,13,13
...,...,...,...
1955,The ATM at Metro bank on High St. Kensington d...,18,18
1956,The ATM won't give back my card,18,18
1957,I was at an ATM and it swallowed my card.,18,18
1958,WTF??? I tried to withdraw some money at a Met...,18,13


In [27]:
test_data_sub['response'] = test_data_sub['response'].astype(int)


/scratch/slurm_tmpdir/job_23779533/ipykernel_169380/3811748138.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data_sub['response'] = test_data_sub['response'].astype(int)


In [28]:
print(classification_report(test_data_sub['label'], test_data_sub['response']))

              precision    recall  f1-score   support

          13       0.83      1.00      0.91        40
          18       1.00      0.80      0.89        40

    accuracy                           0.90        80
   macro avg       0.92      0.90      0.90        80
weighted avg       0.92      0.90      0.90        80



In [21]:
print(classification_report(test_data_sub['label'], test_data_sub['response']))

              precision    recall  f1-score   support

          13       0.58      0.88      0.70        40
          18       0.75      0.38      0.50        40

    accuracy                           0.62        80
   macro avg       0.67      0.62      0.60        80
weighted avg       0.67      0.62      0.60        80



In [13]:
#iterate over the test data and generate the prompt
#surpress warnings
import warnings

warnings.filterwarnings("ignore")

prompt_template = get_prompt_template(test_data_sub, num_shots)

n_classes_list = [5, 10, 20, 30, 40, 50, 77]
num_shots = 5
results = {}
#iterate over the n_classes_list

# import wandb 
# wandb.init(project="llm-banking77")

from sklearn.metrics import classification_report

for n_classes in n_classes_list:
    sampled_classes = train_data['label'].sample(n_classes, random_state=random_seed).values
    train_data_sub = train_data[train_data['label'].isin(sampled_classes)]
    test_data_sub = test_data[test_data['label'].isin(sampled_classes)]
    prompt_template = get_prompt_template(test_data_sub, num_shots)

    for i, row in tqdm.tqdm(test_data_sub.iterrows(),total=len(test_data_sub), desc="Processing data"):
        prompt = prompt_template + f"Text: {row['text']}\n"
        response = llm(prompt=prompt)
        test_data_sub.loc[i, 'response'] = response
    test_data_sub['response'] = test_data_sub['response'].label.astype(int)
    report = classification_report(test_data_sub['label'], test_data_sub['response'], output_dict=True)
    # #print(f"Classification Report: {classification_report}")
    # #save results to dictionary
    # results[n_classes] = {}
    results[n_classes] = {
        'classification_report': report,
        'prompt_template': prompt_template,
        'context_length': len(prompt_template)
    }
    
 

Processing data:   0%|          | 0/200 [00:00<?, ?it/s]


TypeError: __call__() missing 1 required keyword-only argument: 'query'

In [ ]:
import json
with open("../data/results.json", "w") as f:
    json.dump(results, f)